In [115]:
# importing package
import numpy as np
import math
import pandas as pd

# make sample
def MakeSample():
    dataSet = np.array([[0, 1, 1, 'yes'],
                        [0, 1, 0, 'no'],
                        [1, 0, 1, 'no'],
                        [1, 1, 1, 'no'],
                        [0, 1, 0, 'no'],
                        [0, 0, 1, 'no'],
                        [1, 0, 1, 'no'],
                        [1, 1, 0, 'no']])

    labels = ['cartoon', 'winter', 'more than 1 person']
    
    return dataSet, labels

# calculate base entropy
def BaseEnt(dataSet):
    label = dataSet[:, -1] #choose label from dataSet
    prob = []
    BaseEnt = []
    for i in pd.unique(label):
        prob.append(len(label[label == i]) / len(label))

#    for j in range(len(pos)):
#        temp = pos.pop()
#        BaseEnt.append(-temp*(math.log2(temp))-temp*(math.log2(temp)))

    BaseEnt = -prob[0]*math.log2(prob[0])-prob[1]*math.log2(prob[1])

    return label, prob, BaseEnt

# calculate shannon entropy
def Shannon(dataSet):
    label = dataSet[:, -1]
    labelcount = {}
    shannon = 0.0
    
    for i in pd.unique(label):
        labelcount[i] = len(label[label == i])

#    for key in labelcount:
#        prob = labelcount[key]/len(dataSet)
#        shannon.append(-prob*math.log2(prob))
#    np.sum(shannon)

    for key in labelcount:
        prob = labelcount[key]/len(dataSet)
        shannon -= prob*math.log2(prob)
    
    return labelcount, shannon

In [116]:
dataset, labels = MakeSample()
label, prob, baseent = BaseEnt(dataset)
labelcount, shannon = Shannon(dataset)